In [ ]:
import requests
import os
from IPython.display import Markdown, display
from functools import wraps

class DeepSeekAPI:
    def __init__(self, api_key=None):
        self.api_key = api_key or os.getenv('DEEPSEEK_API_KEY')
        self.base_url = "https://api.deepseek.com/v1"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
    
    def chat(self, prompt, model="deepseek-chat", temperature=0.7, max_tokens=1000):
        """Send a chat request to DeepSeek API"""
        endpoint = f"{self.base_url}/chat/completions"
        
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": temperature,
            "max_tokens": max_tokens
        }
        
        response = requests.post(endpoint, headers=self.headers, json=payload)
        response.raise_for_status()
        return response.json()

# Create a magic command wrapper
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def deepseek(line, cell=None):
    """
    IPython magic command for DeepSeek API
    Usage:
    %deepseek What is Python?
    or
    %%deepseek
    Write me a function that...
    """
    api = DeepSeekAPI()
    content = cell if cell else line
    
    try:
        response = api.chat(content)
        # Display as markdown for better formatting
        display(Markdown(response['choices'][0]['message']['content']))
    except Exception as e:
        print(f"Error: {e}")

        class DeepSeekAPI:
    def __init__(self, api_key=None):
        self.api_key = api_key or os.getenv('DEEPSEEK_API_KEY')
        if not self.api_key:
            raise ValueError("DeepSeek API key not found. Set DEEPSEEK_API_KEY environment variable or pass it directly.")
        
        self.base_url = "https://api.deepseek.com/v1"
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        # Cache for storing conversation history
        self.conversation_history = []
    
    def chat(self, prompt, model="deepseek-chat", temperature=0.7, max_tokens=1000, stream=False):
        """Send a chat request to DeepSeek API"""
        self.conversation_history.append({"role": "user", "content": prompt})
        
        endpoint = f"{self.base_url}/chat/completions"
        payload = {
            "model": model,
            "messages": self.conversation_history,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stream": stream
        }
        
        try:
            response = requests.post(endpoint, headers=self.headers, json=payload)
            response.raise_for_status()
            
            if stream:
                return response.iter_lines()
            
            result = response.json()
            self.conversation_history.append({
                "role": "assistant",
                "content": result['choices'][0]['message']['content']
            })
            return result
            
        except requests.exceptions.RequestException as e:
            print(f"API Request Failed: {e}")
            return None
    
    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []

# Enhanced magic command
@register_line_cell_magic
def deepseek(line, cell=None):
    """
    IPython magic command for DeepSeek API
    Options:
        -t or --temperature: Set temperature (0.0 to 1.0)
        -m or --model: Specify model
        -c or --clear: Clear conversation history
    """
    import argparse
    
    parser = argparse.ArgumentParser(description='DeepSeek API Magic Command')
    parser.add_argument('-t', '--temperature', type=float, default=0.7)
    parser.add_argument('-m', '--model', default="deepseek-chat")
    parser.add_argument('-c', '--clear', action='store_true')
    
    try:
        args = parser.parse_known_args(line.split())[0]
    except:
        args = parser.parse_args([])
    
    api = DeepSeekAPI()
    
    if args.clear:
        api.clear_history()
        print("Conversation history cleared.")
        return
    
    content = cell if cell else line
    response = api.chat(content, model=args.model, temperature=args.temperature)
    
    if response:
        display(Markdown(response['choices'][0]['message']['content']))


# Initialize the magic command
get_ipython().register_magic_function(deepseek, 'line_cell')

In [ ]:
## Usage # Single line
%deepseek What is Python?

# Or multi-line
%%deepseek
Write a Python function that:
1. Takes a list of numbers
2. Returns the sum of even numbers
3. Include docstring and type hints